# 11: GNSS Precise Ephemeris & IONEX TEC Maps

This workshop covers using real-world precise ephemeris (SP3) and ionospheric
TEC grids (IONEX) for cm-level satellite positioning and accurate ionospheric
delay modeling.

## Learning Objectives
- Understand SP3 precise ephemeris format and sources
- Work with IONEX TEC maps for ionospheric correction
- Compare broadcast vs precise ephemeris accuracy
- Generate GNSS scenarios with real-world precision
- Interpret satellite clock corrections

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import struct
import os
from datetime import datetime, timedelta

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# Helper: run r4w CLI and capture output
def r4w(*args, features=None):
    cmd = ['cargo', 'run', '--bin', 'r4w']
    if features:
        cmd.extend(['--features', features])
    cmd.append('--')
    cmd.extend(list(args))
    result = subprocess.run(cmd, capture_output=True, text=True, 
                          cwd=os.path.join(os.getcwd(), '..'))
    print(result.stdout)
    if result.returncode != 0:
        print('STDERR:', result.stderr)
    return result

# Helper: read IQ files in different formats
def read_iq(path, fmt='f64'):
    with open(path, 'rb') as f:
        data = f.read()
    if fmt == 'f64':
        samples = struct.unpack(f'<{len(data)//8}d', data)
        return np.array(samples[0::2]) + 1j * np.array(samples[1::2])
    elif fmt in ('f32', 'ettus'):
        samples = struct.unpack(f'<{len(data)//4}f', data)
        return np.array(samples[0::2]) + 1j * np.array(samples[1::2])
    elif fmt == 'sc16':
        samples = struct.unpack(f'<{len(data)//2}h', data)
        iq = np.array(samples[0::2]) + 1j * np.array(samples[1::2])
        return iq / 32768.0  # Normalize to [-1, 1]
    else:
        raise ValueError(f'Unknown format: {fmt}')

## Part 1: SP3 Precise Ephemeris

SP3 (Standard Product 3) files contain precise satellite positions and clock
corrections computed post-facto from ground station observations. Sources:

| Provider | Latency | Accuracy | Update Rate |
|----------|---------|----------|-------------|
| IGS Ultra-Rapid | 3-9 hours | 5 cm | Every 6 hours |
| IGS Rapid | 17-41 hours | 2.5 cm | Daily |
| IGS Final | 12-18 days | <2 cm | Weekly |
| CODE Final | 12-18 days | <2 cm | Daily |

### SP3 File Format

```
#cP2026  1  5  0  0  0.00000000      96 ORBIT IGS14 HLM  IGS
## 2295 259200.00000000   900.00000000 60679 0.0000000000000
+ 85   G01G02G03G04G05G06G07G08G09G10G11G12G13G14G15G16G17
...
*  2026  1  5  0  0  0.00000000
PG01  12345.678901 -23456.789012  34567.890123    123.456789
PG02  ...
```

Each `P` record contains:
- Satellite ID (G01 = GPS PRN 1, E05 = Galileo E05, R24 = GLONASS R24)
- X, Y, Z position in km (ITRF reference frame)
- Clock correction in microseconds

In [ ]:
# SP3 file naming convention:
# COD0OPSFIN_YYYYDDD0000_01D_05M_ORB.SP3.gz
#   COD = CODE analysis center
#   0OPS = Operational product
#   FIN = Final solution
#   YYYY = Year
#   DDD = Day of year
#   01D = 1-day file span
#   05M = 5-minute sample interval

# Example: January 5, 2026 = DOY 005
date = datetime(2026, 1, 5)
doy = date.timetuple().tm_yday
sp3_filename = f"COD0OPSFIN_{date.year}{doy:03d}0000_01D_05M_ORB.SP3.gz"
print(f"SP3 filename for {date.strftime('%Y-%m-%d')}: {sp3_filename}")

# Download URL (CODE FTP server)
ftp_url = f"ftp://ftp.aiub.unibe.ch/CODE/{date.year}/{sp3_filename}"
print(f"Download URL: {ftp_url}")

## Part 2: IONEX TEC Maps

IONEX (IONosphere Map Exchange) files contain global grids of Total Electron
Content (TEC) - the integral of electron density along the signal path.

### TEC Units
- 1 TECU = 10^16 electrons/m^2
- L1 delay ≈ 0.162 meters per TECU
- L5 delay ≈ 0.290 meters per TECU (1/f^2 scaling)

### IONEX Grid Structure
- Latitude: 87.5°N to 87.5°S (2.5° spacing)
- Longitude: 180°W to 180°E (5° spacing)
- Time: Every 1-2 hours (typically 13-25 maps per day)

In [ ]:
# IONEX file naming convention:
# COD0OPSFIN_YYYYDDD0000_01D_01H_GIM.INX.gz
#   GIM = Global Ionosphere Map
#   01H = 1-hour time resolution

ionex_filename = f"COD0OPSFIN_{date.year}{doy:03d}0000_01D_01H_GIM.INX.gz"
print(f"IONEX filename for {date.strftime('%Y-%m-%d')}: {ionex_filename}")

# The TEC value at a given time/location is interpolated from the grid
# Ionospheric delay in meters:
#   delay_L1 = 40.3 * TEC / f_L1^2  (where TEC in el/m^2, f in Hz)
#   delay_L1 ≈ 0.162 * TEC_TECU    (simplified)

# Example TEC values by region and time of day:
tec_examples = {
    'Mid-latitude night': 5,
    'Mid-latitude day': 20,
    'Equatorial night': 15,
    'Equatorial day (peak)': 80,
    'Polar region': 10,
}

print("\nExample TEC values and L1 delays:")
print(f"{'Region/Time':<25} {'TEC (TECU)':<12} {'L1 Delay (m)':<12}")
print("-" * 50)
for region, tec in tec_examples.items():
    delay = 0.162 * tec
    print(f"{region:<25} {tec:<12} {delay:<12.1f}")

## Part 3: Broadcast vs Precise Ephemeris Comparison

| Parameter | Broadcast | SP3 Precise |
|-----------|-----------|-------------|
| Position accuracy | 1-2 m | 2-5 cm |
| Clock accuracy | 2-5 ns | 0.1 ns |
| Availability | Real-time | Post-processed |
| Use case | Navigation | Geodesy, research |

In [ ]:
# Visualize error sources in GNSS positioning
error_sources = {
    'Broadcast ephemeris': 2.0,
    'Satellite clock': 1.5,
    'Ionosphere (uncorrected)': 7.0,
    'Ionosphere (Klobuchar)': 3.5,
    'Ionosphere (IONEX)': 0.5,
    'Troposphere': 0.3,
    'Multipath': 1.0,
    'Receiver noise': 0.3,
}

# Compare standard vs precise
standard_errors = ['Broadcast ephemeris', 'Satellite clock', 'Ionosphere (Klobuchar)', 
                   'Troposphere', 'Multipath', 'Receiver noise']
precise_errors = ['Ionosphere (IONEX)', 'Troposphere', 'Multipath', 'Receiver noise']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Standard processing
std_vals = [error_sources[e] for e in standard_errors]
std_total = np.sqrt(sum(v**2 for v in std_vals))
axes[0].barh(standard_errors, std_vals, color='steelblue')
axes[0].axvline(x=std_total, color='red', linestyle='--', label=f'RSS Total: {std_total:.1f}m')
axes[0].set_xlabel('Error (meters, 1-sigma)')
axes[0].set_title('Standard Processing (Broadcast + Klobuchar)')
axes[0].legend()

# Precise processing
prec_vals = [error_sources[e] for e in precise_errors]
prec_total = np.sqrt(sum(v**2 for v in prec_vals))
axes[1].barh(precise_errors, prec_vals, color='darkgreen')
axes[1].axvline(x=prec_total, color='red', linestyle='--', label=f'RSS Total: {prec_total:.1f}m')
axes[1].set_xlabel('Error (meters, 1-sigma)')
axes[1].set_title('Precise Processing (SP3 + IONEX)')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nPosition error reduction: {std_total:.1f}m → {prec_total:.1f}m ({100*(1-prec_total/std_total):.0f}% improvement)")

## Part 4: Satellite Clock Corrections

SP3 files include satellite clock corrections that account for:
- Atomic clock drift (Rubidium or Cesium oscillators)
- Relativistic effects (velocity and gravitational time dilation)
- Hardware delays in the satellite

Clock corrections are typically in the range of ±100 microseconds and must be
applied to achieve precise positioning.

In [ ]:
# Relativistic effects on GPS satellite clocks
c = 299792458.0  # Speed of light (m/s)
GM = 3.986004418e14  # Earth gravitational parameter
a_gps = 26559700.0  # GPS orbital radius (m)
R_earth = 6371000.0  # Earth radius (m)

# Orbital velocity
v_sat = np.sqrt(GM / a_gps)
print(f"GPS satellite velocity: {v_sat:.0f} m/s = {v_sat/1000:.2f} km/s")

# Special relativistic time dilation (satellite clock runs SLOWER)
gamma = 1 / np.sqrt(1 - (v_sat/c)**2)
sr_effect_per_day = (gamma - 1) * 86400 * 1e6  # microseconds per day
print(f"\nSpecial relativistic effect: {sr_effect_per_day:.1f} μs/day (clock runs slower)")

# General relativistic effect (satellite clock runs FASTER due to weaker gravity)
# Δf/f = -ΔU/c^2 where ΔU is gravitational potential difference
U_sat = -GM / a_gps
U_surface = -GM / R_earth
gr_effect_per_day = (U_surface - U_sat) / c**2 * 86400 * 1e6  # microseconds per day
print(f"General relativistic effect: {gr_effect_per_day:.1f} μs/day (clock runs faster)")

# Net effect
net_effect = gr_effect_per_day - sr_effect_per_day
print(f"\nNet relativistic effect: {net_effect:.1f} μs/day")
print(f"Equivalent range error if uncorrected: {net_effect * c / 1e6:.0f} m/day")

# This is why GPS clocks are intentionally offset by -4.465 parts per billion

## Part 5: Using SP3/IONEX with R4W

The R4W CLI supports SP3 and IONEX files via the `--sp3` and `--ionex` options
(requires the `ephemeris` feature).

```bash
# Generate scenario with precise ephemeris
cargo run --bin r4w --features ephemeris -- gnss scenario \
    --preset open-sky \
    --sp3 /path/to/COD0OPSFIN_20260050000_01D_05M_ORB.SP3 \
    --ionex /path/to/COD0OPSFIN_20260050000_01D_01H_GIM.INX \
    --duration 0.01 \
    --output precise_scenario.iq
```

The satellite status output shows clock corrections for each PRN.

In [ ]:
# Note: This cell requires actual SP3/IONEX files
# Download from CODE FTP or use the r4w ephemeris caching system

# Example output format with clock corrections:
example_output = """
Satellite Status at t=0.000s:
PRN  Signal     El(°)   Az(°)    Range(km)  Doppler(Hz)  C/N0(dB-Hz)  Iono(m)  Tropo(m)  Clk(μs)
G02  GPS-L1CA   45.2    123.4    22345.6      +1234.5       42.3       5.6       4.2      +12.34
G05  GPS-L1CA   23.1    234.5    24567.8       -987.6       38.7       8.9       7.1      -23.45
G10  GPS-L1CA   67.8    345.6    21234.5       +456.7       44.1       3.2       2.8      +34.56
G17  GPS-L1CA   12.3     56.7    25678.9      -2345.6       35.2      12.3      15.4       -5.67
"""
print("Example scenario output with SP3 clock corrections:")
print(example_output)

# The Clk(μs) column shows the satellite clock correction from SP3
# Positive = clock is ahead, negative = clock is behind
# These corrections are applied to the pseudorange calculation

## Part 6: Output Formats

R4W supports multiple IQ output formats for compatibility with different SDR tools:

| Format | Type | Size/Sample | Use Case |
|--------|------|-------------|----------|
| `f64` | Complex float64 | 16 bytes | Maximum precision |
| `f32` | Complex float32 | 8 bytes | General purpose |
| `ettus` | Interleaved float32 | 8 bytes | USRP/GNU Radio |
| `sc16` | Signed int16 | 4 bytes | Compact storage |

In [ ]:
# MATLAB code to read Ettus format:
matlab_code = '''
% Read USRP/Ettus format IQ file in MATLAB
fid = fopen('gnss_scenario.iq', 'rb');
data = fread(fid, inf, 'float32');
fclose(fid);

% Interleaved I/Q -> complex
iq = data(1:2:end) + 1j * data(2:2:end);

% Plot spectrum
Fs = 2.046e6;  % Sample rate
N = length(iq);
f = (-N/2:N/2-1) * Fs / N / 1e3;  % kHz
spectrum = fftshift(fft(iq));
plot(f, 10*log10(abs(spectrum).^2/N));
xlabel('Frequency (kHz)');
ylabel('Power (dB)');
title('GNSS Scenario Spectrum');
'''
print("MATLAB code to read Ettus format:")
print(matlab_code)

# Python/NumPy code:
python_code = '''
import numpy as np

# Read Ettus format
data = np.fromfile('gnss_scenario.iq', dtype=np.float32)
iq = data[0::2] + 1j * data[1::2]

# Read sc16 format
data = np.fromfile('gnss_scenario.iq', dtype=np.int16)
iq = (data[0::2] + 1j * data[1::2]) / 32768.0
'''
print("\nPython/NumPy code:")
print(python_code)

In [ ]:
# File size comparison for different formats
duration_s = 1.0
sample_rate = 4.092e6  # 4.092 MHz (common for GPS L1)
num_samples = int(duration_s * sample_rate)

formats = {
    'f64 (complex128)': 16,
    'f32 (complex64)': 8,
    'ettus (float32 interleaved)': 8,
    'sc16 (int16 interleaved)': 4,
}

print(f"File sizes for {duration_s}s at {sample_rate/1e6:.3f} MHz ({num_samples:,} samples):")
print(f"{'Format':<30} {'Size':<15} {'Relative'}")
print("-" * 55)
for fmt, bytes_per_sample in formats.items():
    size_bytes = num_samples * bytes_per_sample
    size_mb = size_bytes / 1e6
    relative = bytes_per_sample / 16 * 100
    print(f"{fmt:<30} {size_mb:>8.1f} MB     {relative:>5.0f}%")

## Exercises

1. **Download real data**: Fetch SP3 and IONEX files from the CODE FTP server
   for yesterday's date. Generate a scenario and compare satellite positions
   with broadcast ephemeris.

2. **TEC visualization**: Parse an IONEX file and create a world map showing
   TEC distribution at solar noon. Observe the equatorial anomaly.

3. **Clock stability analysis**: Extract clock corrections for a single
   satellite over 24 hours from an SP3 file. Plot the drift and estimate
   the Allan variance.

4. **Dual-frequency correction**: Using the 1/f^2 ionospheric scaling,
   calculate the ionosphere-free combination from L1 and L5 pseudoranges.
   Compare with the IONEX-derived correction.

In [ ]:
# Your code here
